In [1]:
EDGE_SITE = "CHI@Edge"
PROJECT_ID = "Chameleon"

OBSERVER_DEVICE_NAME = "iot-rpi-cm4-02"
OBSERVER_LEASE_NAME = f"observer-{OBSERVER_DEVICE_NAME}"

MEASURED_DEVICE_NAME = "iot-rpi-cm4-01"
MEASURED_LEASE_NAME = f"measured-{MEASURED_DEVICE_NAME}"

SSH_CONNECTION = "cc@129.114.27.80"
SSH_OPTIONS = "-o StrictHostKeyChecking=no -i /id_rsa"

In [2]:
import time

import chi
chi.set("project_name", PROJECT_ID)
chi.use_site(EDGE_SITE)

from chi import lease
from chi import server
from chi import container

Now using CHI@Edge:
URL: https://chi.edge.chameleoncloud.org
Location: University of Chicago, Chicago, Illinois, USA
Support contact: help@chameleoncloud.org


## Reserve observer-device and measured-device

In [3]:
start, end = lease.lease_duration(days=7)

observer_reservation = []
lease.add_device_reservation(observer_reservation, count=1, device_name=OBSERVER_DEVICE_NAME)
try:
    observer_lease = lease.get_lease(OBSERVER_LEASE_NAME)
except:
    observer_lease = lease.create_lease(OBSERVER_LEASE_NAME, observer_reservation, start_date=start, end_date=end)

measured_reservation = []
lease.add_device_reservation(measured_reservation, count=1, device_name=MEASURED_DEVICE_NAME)
try:
    measured_lease = lease.get_lease(MEASURED_LEASE_NAME)
except:
    measured_lease = lease.create_lease(MEASURED_LEASE_NAME, measured_reservation, start_date=start, end_date=end)

lease.wait_for_active(observer_lease["id"])
lease.wait_for_active(measured_lease["id"])

observer_reservation_id = lease.get_lease(OBSERVER_LEASE_NAME)["reservations"][0]["id"]
measured_reservation_id = lease.get_lease(MEASURED_LEASE_NAME)["reservations"][0]["id"]

error: not enough resources available with query {'resource_type': 'device', 'min': 1, 'max': 1, 'resource_properties': '["==", "$name", "iot-rpi-cm4-02"]', 'start_date': datetime.datetime(2023, 4, 24, 21, 25), 'end_date': datetime.datetime(2023, 5, 1, 21, 24), 'project_id': 'a5f0758da4a5404bbfcef0a64206614c', 'count_range': '1-1', 'before_end': 'default', 'on_start': 'default'}
error: not enough resources available with query {'resource_type': 'device', 'min': 1, 'max': 1, 'resource_properties': '["==", "$name", "iot-rpi-cm4-01"]', 'start_date': datetime.datetime(2023, 4, 24, 21, 25), 'end_date': datetime.datetime(2023, 5, 1, 21, 24), 'project_id': 'a5f0758da4a5404bbfcef0a64206614c', 'count_range': '1-1', 'before_end': 'default', 'on_start': 'default'}


TypeError: 'NoneType' object is not subscriptable

Application performance: I am sure this has been done to death but it may be a good thing to do it again to get a baseline: we run 2-3 selected ML models in a CHI@Edge container and measure performance (execution time) as well as their power draw to give people an idea (maybe even a timeline for power draw) – for extra credit if we can do data taking from the camera and running ML on the device that would make it very realistic. We measure on bare metal (user deploys own device) and using CHI@Edge. 


In [4]:
need_to_add_key = False
try:
    observer_container = container.get_container("mppowers-observer")
except:
    observer_container = container.create_container(
        "mppowers-observer", 
        image="mppowers/observer:latest",
        device_profiles=["pi_meter"],
        reservation_id=observer_reservation_id,
        interactive=True,
        exposed_ports=["8000"],
    )
    need_to_add_key = True
    container.wait_for_active(observer_container.uuid)
    
if "No such file" in container.execute(observer_container.uuid, 'ls /id_rsa')["output"]:
    print("Generating SSH key")
    print(container.execute(observer_container.uuid, 'ssh-keygen -t rsa -f /id_rsa -q -P ""')["output"])
    need_to_add_key = True
elif "hello" not in container.execute(observer_container.uuid, f'ssh {SSH_OPTIONS} {SSH_CONNECTION} echo "hello"')["output"]:
    need_to_add_key = True

if need_to_add_key:
    print(f"Please add this key to {SSH_CONNECTION}")
    print(container.execute(observer_container.uuid, 'cat /id_rsa.pub')["output"])
else:
    print("Successfully connected to file transfer node")

Please add this key to cc@129.114.27.80
ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABgQCpkiGjtMTWc2YVfO60CchMJH2yI+AkK+piazOw6TWAfUjkjwOmNfOV/ToZyKFos3IGsu3/PIw/9/CUpvTV1zQnLc+E03Okh96iJJWw9sL+t3p3FOk+8cK19XnQ8bK5kMU+JUshBoYVCl+8sJnIJQmQTRE9HquOP0ho4+FsN+gpqnqOzu69Hnmqe6ipoaZ+kGoIoeWWnhBA+Wcm23U6ujc7Fbuu6p/JU/04+6u59yQ59ulHFWPBeTTTWxyadtENKKZyEOB8FROi0HpaHEdg6NBsjaT0ZfuUUnS0RxG1RN7bPsmWzyeVeukIyIBuIM+qOrRbg0ChrGY6rL3lV3W8yKajbd6YWeO5b06CpAJYEPB20EDJdq/bTrAx6LUQ+2B3KXitJH0NH1Jx6J7tn5zedr9HcX5th+H7G1NGNqb+bac/d68I/a8ito6evgACxA3A6RZiQkAFYZnKdHjL+414Vy7MDA9pDz8esPcZGNCdJY22a/4FBy8ZwNCVptZyY5N66/0= root@zun-6c0a9d85-6ea6-42a1-afe4-4b9203683876-6799cd9d89-xzxl7



In [14]:
try:
    experiment_container = container.get_container("mppowers-experiment")
    container.destroy_container(experiment_container.uuid)
    time.sleep(10)
except:
    pass

# container init
print(container.execute(observer_container.uuid, 'python3 rpc_client.py creating resnet')["output"])
experiment_container = container.create_container(
    "mppowers-experiment", 
    image="mppowers/experimenter_mobile_net_ssd",
    reservation_id=measured_reservation_id,
)
print(container.execute(observer_container.uuid, 'python3 rpc_client.py running resnet')["output"])


In [35]:
# Wait until container is no longer running
while True:
    time.sleep(1)
    experiment_container = container.get_container("mppowers-experiment")
    if experiment_container.status != "Running":
        break

print(experiment_container)

In [42]:
print(container.execute(observer_container.uuid, 'python3 rpc_client.py finished resnet')["output"])

In [50]:
print(container.execute(observer_container.uuid, f'scp {SSH_OPTIONS} /out.log {SSH_CONNECTION}:.')["output"])

In [49]:
print(container.execute(observer_container.uuid, f'tail /out.log ')["output"])

INFO:root:1676389500.6345718,441000.000000,5037.000000,2221317000.000000
INFO:root:1676389500.835866,518800.000000,5031.000000,2610082800.000000
INFO:root:1676389501.0372152,672500.000000,4965.000000,3338962500.000000
INFO:root:1676389501.2380903,732200.000000,4956.000000,3628783200.000000
INFO:root:1676389501.4394538,474000.000000,5041.000000,2389434000.000000
INFO:root:1676389501.6404092,567900.000000,5002.000000,2840635800.000000
INFO:root:1676389501.8413012,732600.000000,4966.000000,3638091600.000000
INFO:root:1676389502.0424285,835900.000000,4932.000000,4122658800.000000
INFO:root:1676389502.2435424,629400.000000,4986.000000,3138188400.000000
INFO:root:1676389502.4444313,594000.000000,4996.000000,2967624000.000000



In [65]:
local_ip = list(observer_container.addresses.items())[0][1][0]["addr"]

In [67]:
local_ip = list(observer_container.addresses.items())
print(container.execute(experiment_container.uuid, f'curl -v {local_ip}:8000')["output"])

curl: (3) [globbing] bad range specification in column 2
curl: (3) [globbing] bad range specification in column 2
* Rebuilt URL to: 192.168.115.211,/
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0* Could not resolve host: 192.168.115.211,
* Closing connection 0
curl: (6) Could not resolve host: 192.168.115.211,
* Rebuilt URL to: version:/
* Could not resolve host: version
* Closing connection 1
curl: (6) Could not resolve host: version
* Rebuilt URL to: 4,/
* Could not resolve host: 4,
* Closing connection 2
curl: (6) Could not resolve host: 4,
* Rebuilt URL to: port:/
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0* Could not resolve host: port
* Closing connection 3
curl: (6) Could not resolve host: port
curl: (3) [globbing] unmatched close brace/bracket in column 37

In [66]:
container.execute(experiment_container.uuid, f'python rpc_client.py {local_ip} test')

{'output': '', 'exit_code': None, 'exec_id': None, 'proxy_url': None}

In [53]:
local_ip


'192.168.115.215'

In [7]:
list(observer_container.addresses.values())[0][0]["addr"]


'192.168.115.211'

In [6]:
experiment_container = container.create_container(
    "mppowers-experiment", 
    image="hello-world",
    reservation_id="751004b6-c930-4325-ab7c-09b17e943d07",
    interactive=True,
)


RuntimeError: Container went in to error state